<a id='sect0'></a>
## <font color='darkblue'>Agenda</font>
* <font size='3ptx'><b><a href='#sect1'>Functions</a></b></font>
* <font size='3ptx'><b><a href='#sect2'>Simple Decorators</a></b></font>
* <font size='3ptx'><b><a href='#sect3'>A Few Real World Examples</a></b></font>
* <font size='3ptx'><b><a href='#sect4'>Fancy Decorators</a></b></font>
* <font size='3ptx'><b><a href='#sect5'>More Real World Examples</a></b></font>
* <font size='3ptx'><b><a href='#sect6'>Homework 5</a></b></font>
* <font size='3ptx'><b><a href='#sect7'>Real World Case (Rohan BCST)</a></b></font>

<a id='sect1'></a>
## <font color='darkblue'>Functions</font>
* <font size='3ptx'><b><a href='#sect1_1'>First-Class Objects</a></b></font>
* <font size='3ptx'><b><a href='#sect1_2'>Inner Functions</a></b></font>
* <font size='3ptx'><b><a href='#sect1_3'>Returning Functions From Functions</a></b></font>

([article source](https://realpython.com/primer-on-python-decorators/#functions)) Before you can understand decorators, you must first understand how functions work. For our purposes, <b>a function returns a value based on the given arguments</b>. Here is a very simple example:
```python
>>> def add_one(number):
...     return number + 1

>>> add_one(2)
3
```

<br/>
<b>In general, functions in Python may also have side effects rather than just turning an input into an output</b>. The <a href='https://realpython.com/python-print/'>print() function</a> is a basic example of this: it returns <a href='https://realpython.com/null-in-python/'>None</a> while having the <b><a href='https://en.wikipedia.org/wiki/Side_effect_(computer_science)'>side effect</a></b> of outputting something to the console. However, to understand decorators, it is enough to think about functions as something that turns given arguments into a value.

<b><font color='darkred'>Note</font></b>: In <a href='https://realpython.com/python-functional-programming/'>functional programming</a>, you work (almost) only with <b>pure functions without side effects</b>. While not a purely functional language, Python supports many of the functional programming concepts, including functions as first-class objects.

<a id='sect1_1'></a>
### <font color='darkgreen'>First-Class Objects</font>
In Python, functions are <b><a href='https://dbader.org/blog/python-first-class-functions'>first-class objects</a></b>. This means that <b>functions can be passed around and used as arguments</b>, just like <a href='https://realpython.com/python-data-types/'>any other object (string, int, float, list, and so on)</a>. Consider the following three functions:

In [1]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

Here, <font color='blue'>say_hello()</font> and <font color='blue'>be_awesome()</font> are regular functions that expect a name given as a string. The <font color='blue'>greet_bob()</font> function however, expects a function as its argument. We can, for instance, pass it the <font color='blue'>say_hello()</font> or the <font color='blue'>be_awesome()</font> function:

In [2]:
greet_bob(say_hello)
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

<a id='sect1_2'></a>
### <font color='darkgreen'>Inner Functions</font>
It’s possible to <a href='https://realpython.com/defining-your-own-python-function/'>define functions</a> inside other functions. Such functions are called <b><font color='darkblue'>inner functions</font></b>. Here’s an example of a function with two inner functions:

In [3]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

What happens when you call the <font color='blue'>parent()</font> function? Think about this for a minute. The output will be as follows:

In [4]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


Furthermore, the inner functions are not defined until the parent function is called. They are locally scoped to <font color='blue'>parent()</font>: they only exist inside the <font color='blue'>parent()</font> function as local <a href='https://realpython.com/python-variables/'>variables</a>. Try calling <font color='blue'>first_child()</font>. You should get an error:
```
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'first_child' is not defined
```

<a id='sect1_3'></a>
### <font color='darkgreen'>Returning Functions From Functions</font>
<b>Python also allows you to use functions as return values</b>. The following example returns one of the inner functions from the outer <font color='blue'>parent()</font> function:

In [5]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

This can be seen in the following example:

In [6]:
import inspect

first = parent(1)
second = parent(2)

print(f"first={first}\nsecond={second}\n")
print(f"first is function? {inspect.isfunction(first)}")

first=<function parent.<locals>.first_child at 0x7fe1843ee510>
second=<function parent.<locals>.second_child at 0x7fe1843ee620>

first is function? True


<a id='sect2'></a>
## <a href='https://realpython.com/primer-on-python-decorators/#simple-decorators'><font color='darkblue'>Simple Decorators</font></a> (<a href='#sect0'>back</a>)
* <font size='3ptx'><b><a href='#sect2_1'>Syntactic Sugar!</a></b></font>
* <font size='3ptx'><b><a href='#sect2_2'>Reusing Decorators</a></b></font>
* <font size='3ptx'><b><a href='#sect2_3'>Decorating Functions With Arguments</a></b></font>
* <font size='3ptx'><b><a href='#sect2_4'>Returning Values From Decorated Functions</a></b></font>
* <font size='3ptx'><b><a href='#sect2_5'>Who Are You, Really?</a></b></font>

<b>Now that you’ve seen that functions are just like any other object in Python, you’re ready to move on and see the magical beast that is the Python decorator</b>. Let’s start with an example:

In [7]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)

Can you guess what happens when you call say_whee()? Try it:

In [8]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


To understand what’s going on here, look back at the previous examples. We are literally just applying everything you have learned so far. <b>The so-called decoration happens at the following line:</b>
```python
say_whee = my_decorator(say_whee)
```

In effect, the name <i>say_whee</i> now points to the <font color='blue'>wrapper()</font> inner function. Remember that you return wrapper as a function when you call <font color='blue'>my_decorator(say_whee)</font>:

In [9]:
print(say_whee)
print(say_whee.__name__)

<function my_decorator.<locals>.wrapper at 0x7fe1843ee9d8>
wrapper


Put simply: <b>decorators wrap a function, modifying its behavior.</b>

Before moving on, let’s have a look at a second example. Because <font color='blue'>wrapper()</font> is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day:

In [10]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)

Depend on your local time, nothing will happen when hour < 7am and hour > 22pm

<a id='sect2_1'></a>
### <font color='darkgreen'>Syntactic Sugar!</font>
The way you decorated <font color='blue'>say_whee()</font> above is a little clunky. First of all, you end up typing the name <i>say_whee</i> three times. In addition, <b>the decoration gets a bit hidden away below the definition of the function</b>.

Instead, Python allows you to <b>use decorators in a simpler way with the <font color='orange'>@</font> symbol</b>, sometimes called the <a href='https://www.python.org/dev/peps/pep-0318/#background'>“pie” syntax</a>. The following example does the exact same thing as the first decorator example:

In [11]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

So, <font color='blue'>@my_decorator</font> is just an easier way of saying <font color='blue'>say_whee = my_decorator(say_whee)</font>. It’s how you apply a decorator to a function.

<a id='sect2_2'></a>
### <font color='darkgreen'>Reusing Decorators</font>
<b>Recall that a decorator is just a regular Python function</b>. All the usual tools for easy reusability are available. Let’s move the decorator to its own module that can be used in many other functions.

* <font color='olive'>my_decorators.py</font>
```python
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice
```

You can now use this new decorator in other files by doing a regular import:

In [12]:
from my_decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")

When you run this example, you should see that the original <font color='blue'>say_whee()</font> is executed twice:

In [13]:
say_whee()

Whee!
Whee!


<font color='violet'><b>Free Bonus</b></font>: <a href='https://realpython.com/primer-on-python-decorators/'>Click here to get access to a free "The Power of Python Decorators"</a> guide that shows you 3 advanced decorator patterns and techniques you can use to write to cleaner and more Pythonic programs.

<a id='sect2_3'></a>
### <font color='darkgreen'>Decorating Functions With Arguments</font>
<b>Say that you have a function that accepts some arguments. Can you still decorate it?</b> Let’s try:

In [14]:
from my_decorators import do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

Unfortunately, running this code raises an error:
```python
>>> greet("World")
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given
```

The problem is that the inner function <font color='blue'>wrapper_do_twice()</font> does not take any arguments, but `name="World"` was passed to it. You could fix this by letting <font color='blue'>wrapper_do_twice()</font> accept one argument, but then it would not work for the <font color='blue'>say_whee()</font> function you created earlier.

The solution is to use <a href='https://realpython.com/python-kwargs-and-args/'><b>\*args and \*\*kwargs</b></a> in the inner wrapper function. Then it will <b>accept an arbitrary number of positional and keyword arguments</b>. Update <font color='olive'>my_decorators.py</font> as follows:
```python
def do_twice_v2(func):
  def wrapper_do_twice_v2(*args, **kwargs):
    func(*args, **kwargs)
    func(*args, **kwargs)
  return wrapper_do_twice
```

The <font color='blue'>wrapper_do_twice_v2()</font> inner function now accepts any number of arguments and passes them on to the function it decorates. Now both your <font color='blue'>say_whee()</font> and <font color='blue'>greet()</font> examples works:

In [15]:
from my_decorators import do_twice_v2

@do_twice_v2
def say_whee():
    print("Whee!")
    
@do_twice_v2
def greet(name):
    print(f"Hello {name}")
    
say_whee()
greet('John')

Whee!
Whee!
Hello John
Hello John


<a id='sect2_4'></a>
### <font color='darkgreen'>Returning Values From Decorated Functions</font>
<b>What happens to the return value of decorated functions?</b> Well, that’s up to the decorator to decide. Let’s say you decorate a simple function as follows:

In [16]:
from my_decorators import do_twice_v2

@do_twice_v2
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

Try to use it:

In [17]:
hi_adam = return_greeting("Adam")
print(hi_adam)  # None

Creating greeting
Creating greeting
None


<font color='darkred'>Oops, your decorator ate the return value from the function.</font>

Because the <font color='blue'>do_twice_wrapper_v2()</font> doesn’t explicitly return a value, the call <font color='blue'>return_greeting("Adam")</font> ended up returning None. <b>To fix this, you need to make sure the wrapper function returns the return value of the decorated function.</b> Update your <font color='olive'>my_decorators.py</font> file:
```python
def do_twice_v3(func):
  def wrapper_do_twice_v3(*args, **kwargs):
    func(*args, **kwargs)
    return func(*args, **kwargs)
  return wrapper_do_twice_v3
```

The return value from the last execution of the function is returned:

In [18]:
from my_decorators import do_twice_v3

@do_twice_v3
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

hi_john = return_greeting("John")
print(hi_john)

Creating greeting
Creating greeting
Hi John


<a id='sect2_5'></a>
### <font color='darkgreen'>Who Are You, Really?</font>
<b>A great convenience when working with Python, especially in the interactive shell, is its powerful introspection ability.</b> <b><a href='https://en.wikipedia.org/wiki/Type_introspection'>Introspection</a></b> is the ability of an object to know about its own attributes at runtime. For instance, a function knows its own name and <a href='https://realpython.com/documenting-python-code/'>documentation</a>:

In [19]:
print(f"print is {print}")
print(f"print's name is {print.__name__}")
help(print)

print is <built-in function print>
print's name is print
Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



The introspection works for functions you define yourself as well:

In [20]:
print(f"say_whee is {say_whee}")
print(f"say_whee's name is {say_whee.__name__}")
help(say_whee)

say_whee is <function do_twice_v2.<locals>.wrapper_do_twice_v2 at 0x7fe184270048>
say_whee's name is wrapper_do_twice_v2
Help on function wrapper_do_twice_v2 in module my_decorators:

wrapper_do_twice_v2(*args, **kwargs)



However, after being decorated, <font color='blue'>say_whee()</font> has gotten very confused about its identity. It now reports being the <font color='blue'>wrapper_do_twice_v2()</font> inner function inside the <font color='blue'>do_twice()</font> decorator. Although technically true, this is not very useful information.

To fix this, decorators should use the <b><a href='https://docs.python.org/3/library/functools.html#functools.wraps'>@functools.wraps</a> decorator, which will preserve information about the original function</b>. Update <font color='olive'>my_decorators.py</font> again:
```python
import functools

def do_twice_v4(func):
    @functools.wraps(func)
    def wrapper_do_twice_v4(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)

    return wrapper_do_twice_v4
```

You do not need to change anything about the decorated <font color='blue'>say_whee()</font> function:

In [21]:
from my_decorators import do_twice_v4

@do_twice_v4
def say_whee():
    print("Whee!")

In [22]:
say_whee()

Whee!
Whee!


In [23]:
print(f"say_whee is {say_whee}")
print(f"say_whee's name is {say_whee.__name__}")
help(say_whee)

say_whee is <function say_whee at 0x7fe184275598>
say_whee's name is say_whee
Help on function say_whee in module __main__:

say_whee()



Much better! Now <font color='olive'>say_whee()</font> is still itself after decoration.

<font color='darkred'><b>Technical Detail:</b></font> The <b><a href='https://docs.python.org/3/library/functools.html#functools.wraps'>@functools.wraps</a></b> decorator uses the function functools.update_wrapper() to update special attributes like \_\_name__ and \_\_doc__ that are used in the introspection.

<a id='sect3'></a>
## <a href='https://realpython.com/primer-on-python-decorators/#a-few-real-world-examples'><font color='darkblue'>A Few Real World Examples</font></a> (<a href='#sect0'>back</a>)
* <font size='3ptx'><b><a href='#sect3_1'>Timing Functions</a></b></font>
* <font size='3ptx'><b><a href='#sect3_2'>Debugging Code</a></b></font>
* <font size='3ptx'><b><a href='#sect3_3'>Registering Plugins</a></b></font>
* <font size='3ptx'><b><a href='#sect3_4'>Is the User Logged In?</a></b></font>

<b>Let’s look at a few more useful examples of decorators</b>. You’ll notice that they’ll mainly follow the same pattern that you’ve learned so far:

```python
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator
```

This formula is a good boilerplate template for building more complex decorators.

<a id='sect3_1'></a>
### <font color='darkgreen'>Timing Functions</font>
Let’s start by creating a <font color='orange'><b>@timer</b></font> decorator. <b>It will measure the time a function takes to execute and print the duration to the console.</b> Here’s the code:
```python
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer
```

Let's check how to apply this decorator with example:

In [24]:
from my_decorators import timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

This decorator works by storing the time just before the function starts running (<font color='brown'>at the line marked # 1</font>) and just after the function finishes (<font color='brown'>at # 2</font>). The time the function takes is then the difference between the two (at # 3). We use the <a href='https://docs.python.org/library/time.html#time.perf_counter'><b>time</b>.perf_counter()</a> function, which does a good job of measuring time intervals. Here are some examples of timings:

In [25]:
waste_some_time(1)
waste_some_time(999)

Finished 'waste_some_time' in 0.0030 secs
Finished 'waste_some_time' in 2.6783 secs


<font color='darkred'><b>Note</b>:</font> The @timer decorator is great if you just want to get an idea about the runtime of your functions. If you want to do more precise measurements of code, you should instead consider the <b><a href='https://docs.python.org/library/timeit.html'>timeit module</a></b> in the standard library. It temporarily disables <a href='https://realpython.com/python-memory-management/#garbage-collection'>garbage collection</a> and runs multiple trials to strip out noise from quick function calls.

<a id='sect3_2'></a>
### <font color='darkgreen'>Debugging Code</font>
The following <b><font color='orange'>@debug</font></b> decorator will print the arguments a function is called with as well as its return value every time the function is called:
```python
def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug
```

The signature is created by joining the <a href='https://dbader.org/blog/python-repr-vs-str'>string representations</a> of all the arguments. The numbers in the following list correspond to the numbered comments in the code:
1. Create a list of the positional arguments. Use <a href='https://docs.python.org/3/library/functions.html#repr'>repr()</a> to get a nice string representing each argument.
2. Create a list of the keyword arguments. The <a href='https://realpython.com/python-f-strings/'>f-string</a> formats each argument as key=value where the `!r` specifier means that <a href='https://docs.python.org/3/library/functions.html#repr'>repr()</a> is used to represent the value.
3. The lists of positional and keyword arguments is joined together to one signature string with each argument separated by a comma.
4. The return value is printed after the function is executed.

Let’s see how the decorator works in practice by applying it to a simple function with one position and one keyword argument:

In [26]:
from my_decorators import debug

@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

Note how the <font color='orange'><b>@debug</b></font> decorator prints the signature and return value of the <fonr color='blue'>make_greeting()</font> function:

In [27]:
make_greeting("Benjamin")

Calling make_greeting('Benjamin')
'make_greeting' returned 'Howdy Benjamin!'


'Howdy Benjamin!'

In [28]:
make_greeting("Richard", age=112)

Calling make_greeting('Richard', age=112)
'make_greeting' returned 'Whoa Richard! 112 already, you are growing up!'


'Whoa Richard! 112 already, you are growing up!'

In [29]:
make_greeting(name="Dorrisile", age=116)

Calling make_greeting(name='Dorrisile', age=116)
'make_greeting' returned 'Whoa Dorrisile! 116 already, you are growing up!'


'Whoa Dorrisile! 116 already, you are growing up!'

<a id='sect3_3'></a>
### <font color='darkgreen'>Registering Plugins</font>
Decorators don’t have to wrap the function they’re decorating. They can also simply register that a function exists and return it unwrapped. This can be used, for instance, to create a light-weight plug-in architecture:

In [30]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

The <b><font color='orange'>@register</font></b> decorator simply stores a reference to the decorated function in the global PLUGINS dict. Note that you do not have to write an inner function or use <b><a href='https://docs.python.org/3/library/functools.html#functools.wraps'>@functools.wraps</a></b> in this example because you are returning the original function unmodified.

The <font color='blue'>randomly_greet()</font> function randomly chooses one of the registered functions to use.

In [31]:
PLUGINS

{'say_hello': <function __main__.say_hello(name)>,
 'be_awesome': <function __main__.be_awesome(name)>}

In [32]:
randomly_greet("Alice")

Using 'say_hello'


'Hello Alice'

The main benefit of this simple plugin architecture is that you do not need to maintain a list of which plugins exist. That list is created when the plugins register themselves. This makes it trivial to add a new plugin: just define the function and decorate it with <b><font color='orange'>@register</font></b>.

If you are familiar with <a href='https://docs.python.org/3/library/functions.html#globals'>globals()</a> in Python, you might see some similarities to how the plugin architecture works. <b><a href='https://docs.python.org/3/library/functions.html#globals'>globals()</a> gives access to all global variables in the current scope, including your plugins</b>:
```python
>>> globals()
{..., # Lots of variables not shown here.
 'say_hello': <function say_hello at 0x7f768eae6730>,
 'be_awesome': <function be_awesome at 0x7f768eae67b8>,
 'randomly_greet': <function randomly_greet at 0x7f768eae6840>}
```

Using the <b><font color='orange'>@register</font></b> decorator, you can create your own curated list of interesting variables, effectively hand-picking some functions from <a href='https://docs.python.org/3/library/functions.html#globals'>globals()</a>.

<a id='sect3_4'></a>
### <font color='darkgreen'>Is the User Logged In?</font>
The final example before moving on to some fancier decorators is commonly used when working with a web framework. In this example, we are using <b><a href='https://realpython.com/tutorials/flask/'>Flask</a></b> to set up a /secret web page that should only be visible to users that are logged in or otherwise authenticated:
```python
from flask import Flask, g, request, redirect, url_for
import functools
app = Flask(__name__)

def login_required(func):
    """Make sure user is logged in before proceeding"""
    @functools.wraps(func)
    def wrapper_login_required(*args, **kwargs):
        if g.user is None:
            return redirect(url_for("login", next=request.url))
        return func(*args, **kwargs)
    return wrapper_login_required

@app.route("/secret")
@login_required
def secret():
    ...
```

While this gives an idea about how to add authentication to your web framework, you should usually not write these types of decorators yourself. For Flask, you can use the <a href='https://flask-login.readthedocs.io/en/latest/#flask_login.login_required'>Flask-Login extension</a> instead, which adds more security and functionality.

<a id='sect4'></a>
## <a href='https://realpython.com/primer-on-python-decorators/#fancy-decorators'><font color='darkblue'>Fancy Decorators</font></a> (<a href='#sect0'>back</a>)
So far, you’ve seen how to create simple decorators. You already have a pretty good understanding of what decorators are and how they work. Feel free to take a break from this article to practice everything you’ve learned.

In the second part of this tutorial, we’ll explore more advanced features, including how to use the following:
* <font size='3ptx'><b><a href='#sect4_1'>Decorating Classes</a></b></font>
* <font size='3ptx'><b><a href='#sect4_2'>Several decorators on one function</a></b></font>
* <font size='3ptx'><b><a href='#sect4_3'>Decorators with arguments</a></b></font>
* <font size='3ptx'><b><a href='#sect4_4'>Decorators that can optionally take arguments</a></b></font>
* <font size='3ptx'><b><a href='#sect4_5'>Stateful decorators</a></b></font>
* <font size='3ptx'><b><a href='#sect4_6'>Classes as decorators</a></b></font>

<a id='sect4_1'></a>
### <font color='darkgreen'>Decorating Classes</font>
There are two different ways you can use decorators on classes. The first one is very close to what you have already done with functions: you can <b>decorate the methods of a class</b>. This was <a href='https://www.python.org/dev/peps/pep-0318/#motivation'>one of the motivations</a> for introducing decorators back in the day.

Some commonly used decorators that are even built-ins in Python are <b><a href='https://docs.python.org/3/library/functions.html#classmethod'>@classmethod</a></b>, <b><a href='https://docs.python.org/3/library/functions.html#staticmethod'>@staticmethod</a></b>, and <b><a href='https://docs.python.org/3/library/functions.html#property'>@property</a></b>. The <b><a href='https://docs.python.org/3/library/functions.html#classmethod'>@classmethod</a></b> and <b><a href='https://docs.python.org/3/library/functions.html#staticmethod'>@staticmethod</a></b> decorators are used to define methods inside a class namespace that are not connected to a particular instance of that class. The <b><a href='https://docs.python.org/3/library/functions.html#property'>@property</a></b> decorator is used to customize <a href='https://docs.python.org/howto/descriptor.html#properties'>getters and setters</a> for class attributes.

The following definition of a Circle class uses the @classmethod, @staticmethod, and @property decorators:
```python
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535
```

Let’s define a class where we decorate some of its methods using the <a href='https://realpython.com/primer-on-python-decorators/#debugging-code'>@debug</a> and <a href='https://realpython.com/primer-on-python-decorators/#timing-functions'>@timer</a> decorators from <a href='https://realpython.com/primer-on-python-decorators/#a-few-real-world-examples'>earlier</a>:

In [33]:
from my_decorators import debug, timer

class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

Using this class, you can see the effect of the decorators:

In [34]:
tw = TimeWaster(1000)

Calling __init__(<__main__.TimeWaster object at 0x7fe184401668>, 1000)
'__init__' returned None


In [35]:
tw.waste_time(999)

Finished 'waste_time' in 0.2599 secs


The other way to use decorators on classes is to <b>decorate the whole class</b>. This is, for example, done in the new <b><a href='https://realpython.com/python-data-classes/'>dataclasses module</a></b> in Python 3.7:
```python
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str
```

The meaning of the syntax is similar to the function decorators. In the example above, you could have done the decoration by writing <font color='blue'>PlayingCard = dataclass(PlayingCard)</font>.

A <a href='https://www.python.org/dev/peps/pep-3129/#rationale'>common use of class decorators</a> is to be a simpler alternative to some use-cases of <b><a href='https://realpython.com/python-metaclasses/'>metaclasses</a></b>. In both cases, <b>you are changing the definition of a class dynamically</b>.

Writing a class decorator is very similar to writing a function decorator. The only difference is that the decorator will receive a class and not a function as an argument. In fact, all the <a href='https://realpython.com/primer-on-python-decorators/#a-few-real-world-examples'>decorators you saw above</a> will work as class decorators. When you are using them on a class instead of a function, their effect might not be what you want. In the following example, the <font color='orange'><b>@timer</b></font> decorator is applied to a class:

In [36]:
from my_decorators import timer

@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

Decorating a class does not decorate its methods. Recall that @timer is just shorthand for <font color='blue'>TimeWaster = timer(TimeWaster)</font>.

In [37]:
tw = TimeWaster(1000)

Finished 'TimeWaster' in 0.0000 secs


In [38]:
tw.waste_time(999)

<a href='https://realpython.com/primer-on-python-decorators/#creating-singletons'>Later</a>, you will see an example defining a proper class decorator, namely <b><font color='orange'>@singleton</font></b>, which ensures that there is only one instance of a class.

<a id='sect4_2'></a>
### <font color='darkgreen'>Nesting Decorators</font>
<b>You can apply several decorators to a function by stacking them on top of each other:</b>

In [39]:
from my_decorators import debug, do_twice_v4

@debug
@do_twice_v4
def greet(name):
    print(f"Hello {name}")

Think about this as the decorators being executed in the order they are listed. In other words, <font color='orange'><b>@debug</b></font> calls <font color='orange'><b>@do_twice_v4</b></font>, which calls <font color='blue'>greet()</font>, or <font color='blue'>debug(do_twice(greet()))</font>:

In [40]:
greet("Eva")

Calling greet('Eva')
Hello Eva
Hello Eva
'greet' returned None


Observe the difference if we change the order of <font color='orange'><b>@debug</b></font> and <font color='orange'><b>@do_twice_v4</b></font>:

In [41]:
from my_decorators import debug, do_twice_v4

@do_twice_v4
@debug
def greet(name):
    print(f"Hello {name}")

In [42]:
greet("Eva")

Calling greet('Eva')
Hello Eva
'greet' returned None
Calling greet('Eva')
Hello Eva
'greet' returned None


<a id='sect4_3'></a>
### <font color='darkgreen'>Decorators With Arguments</font>
<b>Sometimes, it’s useful to pass arguments to your decorators.</b> For instance, <font color='orange'><b>@do_twice_v4</b></font> could be extended to a <font color='orange'><b>@repeat(num_times)</b></font> decorator. The number of times to execute the decorated function could then be given as an argument.

This would allow you to do something like this:
```python
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")
```

So it's execution looks like:
```python
>>> greet("World")
Hello World
Hello World
Hello World
Hello World
```

Think about how you could achieve this.

So far, the name written after the @ has referred to a function object that can be called with another function. To be consistent, you then need <font color='blue'>repeat(num_times=4)</font> to return a function object that can act as a decorator. Luckily, <a href='https://realpython.com/primer-on-python-decorators/#returning-functions-from-functions'>you already know how to return functions!</a> In general, you want something like the following:
```python
def repeat(num_times):
    def decorator_repeat(func):
        ...  # Create and return a wrapper function
    return decorator_repeat
```

Typically, the decorator creates and returns an inner wrapper function, so writing the example out in full will give you an inner function within an inner function. While this might sound like the programming equivalent of the <a href='https://en.wikipedia.org/wiki/Inception'>Inception movie</a>, we’ll untangle it all in a moment:
```python
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat
```

It looks a little messy, but we have only put the same decorator pattern you have seen many times by now inside one additional def that handles the arguments to the decorator. Let’s start with the innermost function:
```python
def wrapper_repeat(*args, **kwargs):
    for _ in range(num_times):
        value = func(*args, **kwargs)
    return value
```

This <font color='blue'>wrapper_repeat()</font> function takes arbitrary arguments and returns the value of the decorated function, <font color='blue'>func()</font>. This wrapper function also contains the loop that calls the decorated function `num_times` times. This is no different from the earlier wrapper functions you have seen, except that it is using the `num_times` parameter that must be supplied from the outside.
<br/><br/>
One step out, you’ll find the decorator function:
```python
def decorator_repeat(func):
    @functools.wraps(func)
    def wrapper_repeat(*args, **kwargs):
        ...
    return wrapper_repeat
```

<br/>
Again, <font color='blue'>decorator_repeat()</font> looks exactly like the decorator functions you have written earlier, except that it’s named differently. That’s because we reserve the base name — <font color='blue'>repeat()</font> — for the outermost function, which is the one the user will call.
<br/><br/>
As you have already seen, the outermost function returns a reference to the decorator function:

```python
def repeat(num_times):
    def decorator_repeat(func):
        ...
    return decorator_repeat
```
<br/><br/>
With everything set up, let’s see if the results are as expected:

In [43]:
from my_decorators import repeat

@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")
    
greet('Selina')

Hello Selina
Hello Selina
Hello Selina
Hello Selina


<a id='sect4_4'></a>
### <font color='darkgreen'>Both Please, But Never Mind the Bread</font>
<b>With a little bit of care, you can also define decorators that can be used both with and without arguments. Most likely, you don’t need this, but it is nice to have the flexibility.</b>

As you saw in the previous section, when a decorator uses arguments, you need to add an extra outer function. The challenge is for your code to figure out if the decorator has been called with or without arguments.

Since the function to decorate is only passed in directly if the decorator is called without arguments, the function must be an optional argument. This means that the decorator arguments must all be specified by keyword. You can enforce this with the special * syntax, which means that <a href='https://www.python.org/dev/peps/pep-3102/'>all following parameters are keyword-only</a>:
```python
def name(_func=None, *, kw1=val1, kw2=val2, ...):  # 1
    def decorator_name(func):
        ...  # Create and return a wrapper function.

    if _func is None:
        return decorator_name                      # 2
    else:
        return decorator_name(_func)               # 3
```

Here, the <i>\_func</i> argument acts as a marker, noting whether the decorator has been called with arguments or not:
1. If name has been called without arguments, the decorated function will be passed in as <i>\_func</i>. If it has been called with arguments, then <i>\_func</i> will be None, and some of the keyword arguments may have been changed from their default values. The * in the argument list means that the remaining arguments can’t be called as positional arguments.
2. In this case, the decorator was called with arguments. Return a decorator function that can read and return a function.
3. In this case, the decorator was called without arguments. Apply the decorator to the function immediately.

<br/>
Using this boilerplate on the <font color='orange'><b>@repeat</b></font> decorator in the previous section, you can write the following:

```python
def repeat(_func=None, *, num_times=2):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)
```

Compare this with the original <font color='orange'><b>@repeat</b></font>. The only changes are the added <i>\_func</i> parameter and the if-else at the end.

<a href='https://github.com/dabeaz/python-cookbook/blob/master/src/9/defining_a_decorator_that_takes_an_optional_argument/example.py'>Recipe 9.6</a> of the excellent <a href='https://realpython.com/asins/1449340377/'>Python Cookbook</a> shows an alternative solution using <a href='https://docs.python.org/library/functools.html#functools.partial'><b>functools</b>.partial()</a>.

These examples show that <font color='orange'><b>@repeat</b></font> can now be used with or without arguments:

In [44]:
@repeat
def say_whee():
    print("Whee!")

@repeat(num_times=3)
def greet(name):
    print(f"Hello {name}")

In [45]:
say_whee()

Whee!
Whee!


In [46]:
greet("Penny")

Hello Penny
Hello Penny
Hello Penny


<a id='sect4_5'></a>
### <font color='darkgreen'>Stateful Decorators</font>
<b>Sometimes, it’s useful to have a decorator that can keep track of state.</b> As a simple example, we will create a decorator that counts the number of times a function is called.

<b><font color='darkred'>Note</font></b>: <a href='https://realpython.com/primer-on-python-decorators/#functions'>In the beginning of this guide</a>, we talked about pure functions returning a value based on given arguments. Stateful decorators are quite the opposite, where the return value will depend on the current state, as well as the given arguments.

In the <a href='https://realpython.com/primer-on-python-decorators/#classes-as-decorators'>next section</a>, you will see how to use classes to keep state. But in simple cases, you can also get away with using <a href='https://www.python.org/dev/peps/pep-0232/'>function attributes</a>:
```python
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls
```

<br/>
The state—the number of calls to the function—is stored in the function attribute <font color='violet'>.num_calls</font> on the wrapper function. Here is the effect of using it:

In [47]:
from my_decorators import count_calls

@count_calls
def say_whee():
    print("Whee!")

In [48]:
say_whee()

Call 1 of 'say_whee'
Whee!


In [49]:
say_whee()

Call 2 of 'say_whee'
Whee!


In [50]:
say_whee.num_calls

2

<a id='sect4_6'></a>
### <font color='darkgreen'>Classes as Decorators</font>
<b>The typical way to maintain state is by using classes</b>. In this section, you’ll see how to rewrite the <font color='orange'><b>@count_calls</b></font> example from the previous section using a class as a decorator.

Recall that the decorator syntax <font color='orange'><b>@my_decorator</b></font> is just an easier way of saying <font color='blue'>func = my_decorator(func)</font>. Therefore, if <i>my_decorator</i> is a class, it needs to take func as an argument in its .\_\_init__() method. Furthermore, the class instance needs to be callable so that it can stand in for the decorated function.

For a class instance to be callable, you implement the special <a href='https://docs.python.org/3/reference/datamodel.html#object.__call__'>.\_\_call__()</a> method:

In [51]:
class Counter:
    def __init__(self, start=0):
        self.count = start

    def __call__(self):
        self.count += 1
        print(f"Current count is {self.count}")

The .\_\_call__() method is executed each time you try to call an instance of the class:

In [52]:
counter = Counter()
counter()  # Current count is 1
counter()  # Current count is 2

Current count is 1
Current count is 2


Therefore, a typical implementation of a decorator class needs to implement .\_\_init__() and .\_\_call__():
```python
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)
```

In [53]:
from my_decorators import CountCalls

@CountCalls
def say_whee():
    print("Whee!")

The .\_\_init__() method must store a reference to the function and can do any other necessary initialization. The .\_\_call__() method will be called instead of the decorated function. It does essentially the same thing as the <font color='blue'>wrapper()</font> function in our earlier examples. Note that you need to use the <a href='https://docs.python.org/3/library/functools.html#functools.update_wrapper'><b>functools</b>.update_wrapper()</a> function instead of <a href='https://docs.python.org/3/library/functools.html#functools.wraps'><b>@functools.wraps</b></a>.

This <font color='orange'><b>@CountCalls</b></font> decorator works the same as the one in the previous section:

In [54]:
say_whee()

Call 1 of 'say_whee'
Whee!


In [55]:
say_whee()

Call 2 of 'say_whee'
Whee!


In [56]:
say_whee.num_calls

2

<a id='sect5'></a>
## <font color='darkblue'>More Real World Examples</font> (<a href='#sect0'>back</a>)
* <font size='3ptx'><b><a href='#sect5_1'>Creating Singletons</a></b></font>
* <font size='3ptx'><b><a href='#sect5_2'>Caching Return Values</a></b></font>
* <font size='3ptx'><b><a href='#sect5_3'>Adding Information About Units</a></b></font>
* <font size='3ptx'><b><a href='#sect5_4'>Validating JSON</a></b></font>

<b>We’ve come a far way now, having figured out how to create all kinds of decorators.</b> Let’s wrap it up, putting our newfound knowledge into creating a few more examples that might actually be useful in the real world.

<a id='sect5_1'></a>
### <font color='darkgreen'>Creating Singletons</font>
<b>A singleton is a class with only one instance.</b> There are several singletons in Python that you use frequently, including `None`, `True`, and `False`. It is the fact that `None` is a singleton that allows you to compare for `None` using the <a href='https://realpython.com/python-is-identity-vs-equality/'>is keyword</a>, like you saw in the <b><a href='https://realpython.com/primer-on-python-decorators/#both-please-but-never-mind-the-bread'>Both Please</a></b> section:
```python
if _func is None:
    return decorator_name
else:
    return decorator_name(_func)
```

<b>Using `is` returns True only for objects that are the exact same instance</b>. The following <font color='orange'><b>@singleton</b></font> decorator turns a class into a singleton by storing the first instance of the class as an attribute. Later attempts at creating an instance simply return the stored instance:

```python
def singleton(cls):
    """Make a class a Singleton class (only one instance)"""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if not wrapper_singleton.instance:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton
```
As you see, this class decorator follows the same template as our function decorators. The only difference is that we are using <i>cls</i> instead of <i>func</i> as the parameter name to indicate that it is meant to be a class decorator.

Let’s see if it works:

In [57]:
from my_decorators import singleton

@singleton
class TheOne:
    pass

In [58]:
first_one = TheOne()
another_one = TheOne()

In [59]:
print(f"id(first_one)={id(first_one)}")
print(f"id(another_one)={id(another_one)}")
print(f"first_one is another_one={first_one is another_one}")

id(first_one)=140606561274232
id(another_one)=140606561274232
first_one is another_one=True


It seems clear that <i>first_one</i> is indeed the exact same instance as <i>another_one</i>.

<b><font color='darkred'>Note</font></b>: Singleton classes are not really used as often in Python as in other languages. The effect of a singleton is usually better implemented as a global variable in a module.

<a id='sect5_2'></a>
### <font color='darkgreen'>Caching Return Values</font>
<b>Decorators can provide a nice mechanism for <a href='https://en.wikipedia.org/wiki/Cache_%28computing%29'>caching</a> and <a href='https://en.wikipedia.org/wiki/Memoization'>memoization</a></b>. As an example, let’s look at a <a href='https://realpython.com/python-thinking-recursively/'>recursive</a> definition of the <a href='https://en.wikipedia.org/wiki/Fibonacci_number'>Fibonacci sequence</a>:

In [60]:
from my_decorators import count_calls

@count_calls
def fibonacci(num):
    if num < 2:
        return num
    
    return fibonacci(num - 1) + fibonacci(num - 2)

While the implementation is simple, its runtime performance is terrible:

In [61]:
fibonacci(5)

Call 1 of 'fibonacci'
Call 2 of 'fibonacci'
Call 3 of 'fibonacci'
Call 4 of 'fibonacci'
Call 5 of 'fibonacci'
Call 6 of 'fibonacci'
Call 7 of 'fibonacci'
Call 8 of 'fibonacci'
Call 9 of 'fibonacci'
Call 10 of 'fibonacci'
Call 11 of 'fibonacci'
Call 12 of 'fibonacci'
Call 13 of 'fibonacci'
Call 14 of 'fibonacci'
Call 15 of 'fibonacci'


5

In [62]:
fibonacci.num_calls

15

<b>To calculate the tenth Fibonacci number, you should really only need to calculate the preceding Fibonacci numbers, but this implementation somehow needs a whopping 15 calculations</b>. It gets worse quickly: 21891 calculations are needed for <font color='blue'>fibonacci(20)</font> and almost 2.7 million calculations for the 30th number. This is because the code keeps recalculating Fibonacci numbers that are already known.

The usual solution is to implement Fibonacci numbers using a for loop and a lookup table. However, simple caching of the calculations will also do the trick:
```python
def cache(func):
    """Keep a cache of previous function calls"""
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = dict()
    return wrapper_cache
```

The cache works as a lookup table, so now <font color='blue'>fibonacci()</font> only does the necessary calculations once:

In [63]:
from my_decorators import cache, count_calls

@count_calls
@cache
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

In [64]:
fibonacci(5)

Call 1 of 'fibonacci'
Call 2 of 'fibonacci'
Call 3 of 'fibonacci'
Call 4 of 'fibonacci'
Call 5 of 'fibonacci'
Call 6 of 'fibonacci'
Call 7 of 'fibonacci'
Call 8 of 'fibonacci'
Call 9 of 'fibonacci'


5

In [65]:
fibonacci.num_calls = 0
fibonacci(20)

Call 1 of 'fibonacci'
Call 2 of 'fibonacci'
Call 3 of 'fibonacci'
Call 4 of 'fibonacci'
Call 5 of 'fibonacci'
Call 6 of 'fibonacci'
Call 7 of 'fibonacci'
Call 8 of 'fibonacci'
Call 9 of 'fibonacci'
Call 10 of 'fibonacci'
Call 11 of 'fibonacci'
Call 12 of 'fibonacci'
Call 13 of 'fibonacci'
Call 14 of 'fibonacci'
Call 15 of 'fibonacci'
Call 16 of 'fibonacci'
Call 17 of 'fibonacci'
Call 18 of 'fibonacci'
Call 19 of 'fibonacci'
Call 20 of 'fibonacci'
Call 21 of 'fibonacci'
Call 22 of 'fibonacci'
Call 23 of 'fibonacci'
Call 24 of 'fibonacci'
Call 25 of 'fibonacci'
Call 26 of 'fibonacci'
Call 27 of 'fibonacci'
Call 28 of 'fibonacci'
Call 29 of 'fibonacci'
Call 30 of 'fibonacci'
Call 31 of 'fibonacci'


6765

In the standard library, a <b><a href='https://realpython.com/lru-cache-python/'>Least Recently Used (LRU)</a></b> cache is available as <a href='https://docs.python.org/library/functools.html#functools.lru_cache'><b>@functools</b>.lru_cache</a>. This decorator has more features than the one you saw above. You should use <a href='https://docs.python.org/library/functools.html#functools.lru_cache'><b>@functools</b>.lru_cache</a> instead of writing your own cache decorator:

In [66]:
import functools

@functools.lru_cache(maxsize=4)
def fibonacci(num):
    print(f"Calculating fibonacci({num})")
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

<b>The <font color='violet'>maxsize</font> parameter specifies how many recent calls are cached. The default value is 128, but you can specify <font color='blue'>maxsize=None</font> to cache all function calls.</b> However, be aware that this can cause memory problems if you are caching many large objects.

You can use the <font color='blue'>.cache_info()</font> method to see how the cache performs, and you can tune it if needed. In our example, we used an artificially small maxsize to see the effect of elements being removed from the cache:

In [67]:
fibonacci(10)

Calculating fibonacci(10)
Calculating fibonacci(9)
Calculating fibonacci(8)
Calculating fibonacci(7)
Calculating fibonacci(6)
Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)


55

In [68]:
fibonacci.cache_info()

CacheInfo(hits=8, misses=11, maxsize=4, currsize=4)

<a id='sect5_3'></a>
### <font color='darkgreen'>Adding Information About Units</font>
The following example is somewhat similar to the <a href='https://realpython.com/primer-on-python-decorators/#registering-plugins'>Registering Plugins</a> example from earlier, in that it does not really change the behavior of the decorated function. Instead, it simply adds <font color='violet'>unit</font> as a function attribute:
```python
def set_unit(unit):
    """Register a unit on a function"""
    def decorator_set_unit(func):
        func.unit = unit
        return func
    return decorator_set_unit
```

The following example calculates the volume of a cylinder based on its radius and height in centimeters:

In [69]:
from my_decorators import set_unit
import math

@set_unit(unit="cm^3")
def volume(radius, height):
    return math.pi * radius**2 * height

This <font color='violet'>.unit</font> function attribute can later be accessed when needed:

In [70]:
v = volume(3, 5)
print(f"{v} {volume.unit}")

141.3716694115407 cm^3


Note that you could have achieved something similar using <a href='https://www.python.org/dev/peps/pep-3107/'>function annotations</a>:
```python
import math

def volume(radius, height) -> "cm^3":
    return math.pi * radius**2 * height
```

However, since annotations are <a href='https://www.python.org/dev/peps/pep-0484/'>used for type hints</a>, it would be hard to combine such units as annotations with <a href='https://realpython.com/python-type-checking/#static-type-checking'>static type checking</a>.

Units become even more powerful and fun when connected with a library that can convert between units. One such library is <b><a href='http://pint.readthedocs.io/'>pint</a></b>. With pint installed (<font color='blue'>pip install Pint</font>), you can for instance convert the volume to cubic inches or gallons:
```python
>>> import pint
>>> ureg = pint.UnitRegistry()
>>> vol = volume(3, 5) * ureg(volume.unit)

>>> vol
<Quantity(141.3716694115407, 'centimeter ** 3')>

>>> vol.to("cubic inches")
<Quantity(8.627028576414954, 'inch ** 3')>

>>> vol.to("gallons").m  # Magnitude
0.0373464440537444
```

You could also modify the decorator to return a pint <b><a href='https://pint.readthedocs.io/en/latest/tutorial.html'>Quantity</a></b> directly. Such a <b><a href='https://pint.readthedocs.io/en/latest/tutorial.html'>Quantity</a></b> is made by multiplying a value with the unit. In pint, units must be looked up in a <b>UnitRegistry</b>. The registry is stored as a function attribute to avoid cluttering the namespace:
```python
def use_unit(unit):
    """Have a function return a Quantity with given unit"""
    use_unit.ureg = pint.UnitRegistry()
    def decorator_use_unit(func):
        @functools.wraps(func)
        def wrapper_use_unit(*args, **kwargs):
            value = func(*args, **kwargs)
            return value * use_unit.ureg(unit)
        return wrapper_use_unit
    return decorator_use_unit
```

With the <font color='orange'><b>@use_unit</b></font> decorator, converting units is practically effortless:

In [71]:
from my_decorators import use_unit

@use_unit("meters per second")
def average_speed(distance, duration):
    return distance / duration

In [72]:
bolt = average_speed(100, 9.58)
bolt

10.438413361169102 <Unit('meter / second')>

In [73]:
bolt.to("km per hour")

37.578288100208766 <Unit('kilometer / hour')>

In [74]:
bolt.to("mph").m  # Magnitude

23.35006567906474

<a id='sect5_4'></a>
### <font color='darkgreen'>Validating JSON</font>
Let’s look at one last use case. Take a quick look at the following <b><a href='https://realpython.com/tutorials/flask/'>Flask</a></b> route handler:
```python
@app.route("/grade", methods=["POST"])
def update_grade():
    json_data = request.get_json()
    if "student_id" not in json_data:
        abort(400)
    # Update database
    return "success!"
```

<br/>
<b>Here we ensure that the key <i>student_id</i> is part of the request. Although this validation works, it really does not belong in the function itself.</b> Plus, perhaps there are other routes that use the exact same validation. So, let’s keep it <a href='https://en.wikipedia.org/wiki/Don%27t_repeat_yourself'>DRY</a> and abstract out any unnecessary logic with a decorator. The following <font color='orange'><b>@validate_json</b></font> decorator will do the job:

```python
from flask import Flask, request, abort
import functools
app = Flask(__name__)

def validate_json(*expected_args):                  # 1
    def decorator_validate_json(func):
        @functools.wraps(func)
        def wrapper_validate_json(*args, **kwargs):
            json_object = request.get_json()
            for expected_arg in expected_args:      # 2
                if expected_arg not in json_object:
                    abort(400)
            return func(*args, **kwargs)
        return wrapper_validate_json
    return decorator_validate_json
```

<br/>
In the above code, the decorator takes a variable length list as an argument so that we can pass in as many string arguments as necessary, each representing a key used to validate the <a href='https://realpython.com/python-json/'>JSON</a> data:

1. The list of keys that must be present in the JSON is given as arguments to the decorator.
2. The wrapper function validates that each expected key is present in the JSON data.

<br/>
The route handler can then focus on its real job—updating grades—as it can safely assume that JSON data are valid:

```python
@app.route("/grade", methods=["POST"])
@validate_json("student_id")
def update_grade():
    json_data = request.get_json()
    # Update database.
    return "success!"
```

<a id='sect6'></a>
## <font color='darkblue'>Homework 5</font> (<a href='#sect0'>back</a>)
<b><a href='https://en.wikipedia.org/wiki/Facade_pattern'>Facade</a></b> 是一個相當常見的 design pattern, 用圖是來封裝複雜的thired party APIs 呼叫流程與繁瑣的套件引用過程, 並以定義了 <b><font size='3ptx'>"統一且不變的 API 函數"</font></b> 提供給外部的程式使用. <b>如此日後就算這些 third party 的使用方法或是 API 的規格有任何變動, 外部使用程式因為是面對統一的 API 函數, 所以不會受到影響</b>. 底下是 Wiki page 的介紹:
> The facade pattern (also spelled façade) is a software-design pattern commonly used in object-oriented programming. Analogous to a facade in architecture, a facade is an object that serves as a front-facing interface masking more complex underlying or structural code. 

<br/>

接著是 Facade 的 class diagram 與 Sequence diagram 範例:
![facade_diagrams](images/Facade_diagrams.PNG)
<br/>

接著在這個 Homework 中, 我們有兩個外部的模組 <font color='olive'>greeting.py</font> 與 <font color='olive'>bye.py</font> 定義如下:

* <font color='olive'><b>greeting.py</b></font>:

```python
def say_hi(name):
  return f"Hi {name} ^^."


def say_hello(name):
  return f"Hello, {name}~"


def say_aloha(name):
  return f"Aloha, {name}!"
```

* <font color='olive'><b>bye.py</b></font>:

```python
def say_goodbye(name):
  return f"Goodbye, {name}!"


def say_seeu(name):
  return f"See you, {name}~"


def say_takecare(name):
  return f"Take care! {name}."
```


<br/>
同時根據上面的 Facade pattern, 我們也定義了一個統一的介面 <font color='olive'>api_facade.py</font> 以提供外部程式使用:

* <font color='olive'><b>api_facade.py</b></font>:

```python
PUBLIC_API_STORES = {}
''' Key as api name; value as api object'''

GREET_API_NAME = 'greet_api'
''' Name of greeting API '''

BYE_API_NAME = 'bye_api'
''' Name of API to say bye'''


def regr_greet_api(name=GREET_API_NAME):
  pass


def regr_bye_api(name=BYE_API_NAME):
  pass
```

這個 Homework 便是需要你幫忙實作上面 <font color='blue'>regr_greet_api</font> 與 <font color='blue'>regr_bye_api</font> 兩個 decorators, 所以我們可以到 <font color='olive'>greeting.py</font> 與 <font color='olive'>bye.py</font> 模組註冊我們想用的 greeting 與 bye APIs. 例如我們可以如下註冊:


In [75]:
import api_facade
from api_facade import regr_greet_api, regr_bye_api

@regr_greet_api
def my_hello(name):
    return f"Hello, {name}~"

@regr_bye_api(api_name='test_bye')
def my_takecare(name):
    return f"Take care! {name}."

如此我們便可以如下使用註冊的 public API:

In [76]:
def do_work(name='Selina', greet_api_name=api_facade.GREET_API_NAME, bye_api_name=api_facade.BYE_API_NAME):
    print(api_facade.PUBLIC_API_STORES[greet_api_name](name))
    print(api_facade.PUBLIC_API_STORES[bye_api_name](name))

In [77]:
do_work(bye_api_name='test_bye')

Hello, Selina~
Take care! Selina.


我們可以動態的改變 greeting, bye 的作法而不會動到我們自己定義的 <font color='blue'>do_work</font> 函式, 例如:

In [78]:
@regr_greet_api
def greet_by_chinese(name):
    return f"早安, {name}~"

@regr_bye_api
def bye_by_chinese(name):
    return f"再見, {name}. ^^"

In [79]:
do_work()

早安, Selina~
再見, Selina. ^^


<a id='sect7'></a>
## <font color='darkblue'>Real World Case - Rohan BCST </font> (<a href='#sect0'>back</a>)
在模組 [`cmd_get_test_function_by_name.py`](https://team.git.corp.google.com/android-qmc-pqm-bt/BTAutomation/+/refs/heads/master/commands/cmd_get_test_function_by_name.py) 代碼 86-88 註解中:
```
  TODO(johnkclee): A long list of switcher is a bad smell.
    We can let each testing module to provide a dict for switcher
    to expand with so to hugely reduce the code here.
```

指出了問題, 在這個模組中, 我們定義了測試函數的名稱與物件到屬性 `self.switcher` 中:

```python
    # TBD: Define testing function for Rohan
    self.switcher = {
      'pick_up_call_from_headset': bt_pick_up_call_from_headset,
      'hang_up_call_from_headset': bt_hang_up_call_from_headset,
      'trigger_redial_from_headset': bt_trigger_redial_from_headset,
      ...
      'volume_down_from_dut': bt_volume_down_from_dut,
      'phone_earpiece_on': bt_phone_earpiece_on,
    }
```

也就是日後如果我們要定義新的測試函數, 必須有以下兩個動作:
* 在對應測試模組中定義測試函數 e.g.: 在 `a2dp_test_utils.py` 中定義 `launch_abc_check`:
```python
def launch_abc_check(device):
    # TBD
    pass
```
* 在 [`cmd_get_test_function_by_name.py`](https://team.git.corp.google.com/android-qmc-pqm-bt/BTAutomation/+/refs/heads/master/commands/cmd_get_test_function_by_name.py) 中的 `self.switcher` 加入一筆新的紀錄. e.g.:
```python
    self.switcher = {
        ...
        'launch_abc_check_from_rohan': launch_abc_check,
        ...
    }
```

這某種程度違反了 <b><a href='https://en.wikipedia.org/wiki/Single-responsibility_principle'>SRP</a></b> (Single Responsibility Principle). 模組 [`cmd_get_test_function_by_name.py`](https://team.git.corp.google.com/android-qmc-pqm-bt/BTAutomation/+/refs/heads/master/commands/cmd_get_test_function_by_name.py) 事實上提供了主程式來根據測試函數名稱來查找對應的測試函數物件, 當我們定義新的測試函數時, 理論上不應該需要修改到此模組!

這裡要請妳利用在 <b><a href='#sect5'>Homework5</a></b> 使用到的技巧, 應用到這個問題上, 讓我們在定義新的測試函數時, 可以不需要頻繁的修改 模組 [`cmd_get_test_function_by_name.py`](https://team.git.corp.google.com/android-qmc-pqm-bt/BTAutomation/+/refs/heads/master/commands/cmd_get_test_function_by_name.py), 而只需要在測試模組上定義新的測試函數時, 加上 decorator <b><font color='orange'>@regr_tf</font></b> 時, 即可將該測試函數動態註冊給 [`cmd_get_test_function_by_name.py`](https://team.git.corp.google.com/android-qmc-pqm-bt/BTAutomation/+/refs/heads/master/commands/cmd_get_test_function_by_name.py) 模組:
```python
@regr_tf
def launch_abc_check(device):
    # TBD
    pass
```